# Day 01 - bigram approach

In [9]:
NAMES_PATH = "/Users/mikolaj/Developer/powielacz/data/names.txt"

In [10]:
words = open(NAMES_PATH, "r").read().splitlines()
print(words[:10])
print(len(words))

['Egon', 'Dymitr', 'Rudolf', 'Ingrid', 'Berta', 'Aleksander', 'Adam', 'Gerhard', 'Balbina', 'Teresa']
600


In [11]:
min([len(word) for word in words])

3

In [12]:
max([len(word) for word in words])

11

In [16]:
for w in words[:3]:
    chs = ["<S>"] + list(w) + ["<E>"]
    for ch1, ch2 in zip(chs, chs[1:]):
        print(ch1, ch2)

<S> E
E g
g o
o n
n <E>
<S> D
D y
y m
m i
i t
t r
r <E>
<S> R
R u
u d
d o
o l
l f
f <E>
